# Imports

In [ ]:
!pip list

In [ ]:
import torch
from torch.amp import autocast, GradScaler
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms as T
import torchvision.transforms.functional as F
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.ops import MultiScaleRoIAlign
import os
import numpy as np
import matplotlib.pyplot as plt
import json
from PIL import Image, ImageDraw
import contextlib
import io
import time
from datetime import datetime
from tqdm import tqdm
from torchvision.transforms import ColorJitter, GaussianBlur
import random
import matplotlib.patches as patches
import albumentations as A

In [ ]:
%pip install -q -U albumentations

In [ ]:
#check if cuda is available
torch.cuda.is_available()

In [ ]:
class HazmatDataset(Dataset):
    def __init__(self, data_dir, annotations_file, transforms=None):
        self.data_dir = data_dir
        self.transforms = transforms

        # Load annotations
        with open(annotations_file) as f:
            data = json.load(f)

        self.images = {img['id']: img for img in data['images']}
        self.annotations = data['annotations']

        # Create image_id to annotations mapping
        self.img_to_anns = {}
        for ann in self.annotations:
            img_id = ann['image_id']
            if img_id not in self.img_to_anns:
                self.img_to_anns[img_id] = []
            self.img_to_anns[img_id].append(ann)

        self.ids = list(self.images.keys())

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        img_info = self.images[img_id]

        # Load image
        img_path = os.path.join(self.data_dir, 'images', img_info['file_name'])
        img = Image.open(img_path).convert('RGB')

        # Get annotations
        anns = self.img_to_anns.get(img_id, [])

        boxes = []
        labels = []
        areas = []
        iscrowd = []

        for ann in anns:
            bbox = ann['bbox']
            # Convert [x, y, w, h] to [x1, y1, x2, y2]
            x_min = bbox[0]
            y_min = bbox[1]
            x_max = bbox[0] + bbox[2]
            y_max = bbox[1] + bbox[3]

            # Filter out degenerate boxes
            if x_max > x_min and y_max > y_min:
                boxes.append([x_min, y_min, x_max, y_max])
                labels.append(ann['category_id'])
                areas.append(ann['area'])
                iscrowd.append(ann['iscrowd'])

        # Convert to tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        areas = torch.as_tensor(areas, dtype=torch.float32)
        iscrowd = torch.as_tensor(iscrowd, dtype=torch.int64)

        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([img_id]),
            'area': areas,
            'iscrowd': iscrowd
        }

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.ids)


# Functions

In [ ]:
class ToTensor(object):
    def __call__(self, image, target):
        # Convert PIL image to tensor
        image = F.to_tensor(image)
        return image, target



def collate_fn(batch):
    return tuple(zip(*batch))

def train_one_epoch(model, optimizer, data_loader, device, scaler):
    model.train()
    total_loss = 0
    total_classifier_loss = 0
    total_box_reg_loss = 0
    total_objectness_loss = 0
    total_rpn_box_reg_loss = 0

    # Voeg tqdm toe om de voortgang te tonen
    progress_bar = tqdm(data_loader, desc="Training", leave=True)
    
    for images, targets in progress_bar:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Wrap the forward pass in autocast
        with autocast(device_type='cuda'):
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        # Scale the loss and call backward
        scaler.scale(losses).backward()
        # Unscales the gradients and calls or skips optimizer.step()
        scaler.step(optimizer)
        # Updates the scale for next iteration
        scaler.update()

        # Bereken de totalen
        total_loss += losses.item()
        total_classifier_loss += loss_dict['loss_classifier'].item()
        total_box_reg_loss += loss_dict['loss_box_reg'].item()
        total_objectness_loss += loss_dict['loss_objectness'].item()
        total_rpn_box_reg_loss += loss_dict['loss_rpn_box_reg'].item()

        # Update tqdm-balk
        progress_bar.set_postfix({
            "Loss": f"{losses.item():.4f}",
            "Classifier": f"{loss_dict['loss_classifier'].item():.4f}",
            "BoxReg": f"{loss_dict['loss_box_reg'].item():.4f}",
        })

    avg_loss = total_loss / len(data_loader)
    avg_classifier_loss = total_classifier_loss / len(data_loader)
    avg_box_reg_loss = total_box_reg_loss / len(data_loader)
    avg_objectness_loss = total_objectness_loss / len(data_loader)
    avg_rpn_box_reg_loss = total_rpn_box_reg_loss / len(data_loader)

    return avg_loss, avg_classifier_loss, avg_box_reg_loss, avg_objectness_loss, avg_rpn_box_reg_loss



# Load ground truth annotations
coco_val = COCO('data/data_faster_rcnn/val/annotations/instances_val.json')

# Prepare predictions in COCO format
# Assuming you have a function to convert model outputs to COCO format
# Conversion to COCO Format
def convert_to_coco_format(outputs, image_ids):
    coco_results = []
    for output, image_id in zip(outputs, image_ids):
        boxes = output['boxes'].cpu().numpy()
        scores = output['scores'].cpu().numpy()
        labels = output['labels'].cpu().numpy()
        
        for box, score, label in zip(boxes, scores, labels):
            coco_results.append({
                'image_id': image_id,
                'category_id': int(label),
                'bbox': [box[0], box[1], box[2] - box[0], box[3] - box[1]],
                'score': float(score)
            })
    return coco_results

# Validation Function
def validate(model, data_loader, coco_gt, device):
    model.eval()
    results = []

    # Add tqdm
    progress_bar = tqdm(data_loader, desc="Validation", leave=True)

    with torch.no_grad():
        for images, targets in progress_bar:
            images = list(image.to(device) for image in images)
            outputs = model(images)
            
            image_ids = [target['image_id'].item() for target in targets]
            coco_results = convert_to_coco_format(outputs, image_ids)
            results.extend(coco_results)

            # Update tqdm-bar
            progress_bar.set_postfix({"Processed": len(results)})

    if not results:
        print("No predictions generated. Skipping evaluation.")
        return [0.0] * 6  # Return dummy metrics for empty results

    # Suppress COCOeval output
    with contextlib.redirect_stdout(io.StringIO()):
        coco_dt = coco_gt.loadRes(results)
        coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
        coco_eval.evaluate()
        coco_eval.accumulate()
        coco_eval.summarize()

    return coco_eval.stats


# Custom backbone to return a dictionary of feature maps
class BackboneWithChannels(torch.nn.Module):
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
    def forward(self, x):
        x = self.backbone(x)
        return {'0': x}
    
# Function to create a subset of the dataset
def create_subset(dataset, percentage):
    """
    Create a subset of the dataset based on the given percentage.
    
    Parameters:
    - dataset: The full dataset.
    - percentage: The fraction of the dataset to use (value between 0.0 and 1.0).
    
    Returns:
    - subset: A subset of the dataset containing the specified percentage of data.
    """
    if not (0.0 < percentage <= 1.0):
        raise ValueError("Percentage must be between 0.0 and 1.0.")
    
    # Determine the subset size
    total_samples = len(dataset)
    subset_size = int(total_samples * percentage)
    
    # Shuffle and select a random subset of indices
    indices = list(range(total_samples))
    random.shuffle(indices)
    subset_indices = indices[:subset_size]
    
    return Subset(dataset, subset_indices)
import os
import re

def sanitize_directory_name(name):
    """Vervang ongeldige tekens in mapnamen voor Windows"""
    # Lijst van verboden tekens: < > : " / \ | ? *
    return re.sub(r'[<>:"/\\|?*]', '-', name)

def create_directory(base_path="data/models/"):
    """
    Create a directory inside the base path named 'faster-rcnn-finetuned-{date}' 
    to store models and logs. The name includes the current date and time in the format 'DD-MM-YYYY HH:MM:SS'.

    Parameters:
    - base_path (str): Base directory where the new directory will be created.

    Returns:
    - directory_path (str): Full path to the created directory.
    """
    # Get the current date and time
    current_time = datetime.now().strftime("%d-%m-%Y_%H:%M:%S")
    
    # Define the full directory path
    directory_name = f"faster-rcnn-finetuned-{current_time}"
    directory_name = sanitize_directory_name(directory_name)
    directory_path = os.path.join(base_path, directory_name)

    # Create the directory
    os.makedirs(directory_path, exist_ok=True)
    
    print(f"Directory created: {directory_path}")
    return directory_path

def train_model(directory, model, optimizer, train_loader, device, train_metrics_list, best_val_map, lr_scheduler, val_loader, coco_val, scaler, epoch):
    
    epoch+=1
    # Start the timer
    start_time = time.time()
    
    # Train for one epoch
    train_loss, train_classifier_loss, train_box_reg_loss, train_objectness_loss, train_rpn_box_reg_loss = train_one_epoch(
        model, optimizer, train_loader, device, scaler)
    
    # Validate and get all COCO-metrics
    val_metrics = validate(model, val_loader, coco_val, device)
    val_map = val_metrics[0]  # mAP@IoU=0.50:0.95
    
    # Stop the timer
    end_time = time.time()
    elapsed_time = end_time - start_time
    minutes, seconds = divmod(elapsed_time, 60)
    
    # Obtain the current learning rate
    current_lr = optimizer.param_groups[0]['lr']
    
    # Prepare data for logging
    data = {
        "epoch": epoch,
        "time_elapsed": (int(minutes), int(seconds)),
        "learning_rate": current_lr,
        "train_loss": train_loss,
        "classifier_loss": train_classifier_loss,
        "box_reg_loss": train_box_reg_loss,
        "objectness_loss": train_objectness_loss,
        "rpn_box_reg_loss": train_rpn_box_reg_loss,
        "val_metrics": val_metrics
    }
    
    # Append current epoch data to metrics list
    train_metrics_list.append(data)
    
    # Print summary for this epoch
    print(f"📊 Epoch {epoch} | ⏳ Time: {int(minutes)}m {int(seconds)}s | 🔄 LR: {current_lr:.6f}")
    print(f"📉 Train Loss: {train_loss:.4f} | 🎯 Classifier: {train_classifier_loss:.4f} | 📦 Box Reg: {train_box_reg_loss:.4f}")
    print(f"🔍 Objectness: {train_objectness_loss:.4f} | 🗂️ RPN Box Reg: {train_rpn_box_reg_loss:.4f}")
    print(f"🧪 mAP | 🟢 mAP@IoU=0.50:0.95: {val_metrics[0]:.4f} | 🔵 mAP@IoU=0.50: {val_metrics[1]:.4f} | 🟣 mAP@IoU=0.75: {val_metrics[2]:.4f}")
    print(f"📏 Small mAP: {val_metrics[3]:.4f} | 📐 Medium mAP: {val_metrics[4]:.4f} | 📏 Large mAP: {val_metrics[5]:.4f}")
    
    # Save epoch data to a log file
    save_epoch_data(directory, data)
    
    # Update learning rate
    lr_scheduler.step()
    
    # Save the latest checkpoint with all metrics
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'val_map': val_map,
        'train_metrics_list': train_metrics_list  # Save all metrics
    }
    torch.save(checkpoint, os.path.join(directory, "latest_model.pth"))
    
    # Save the best model if the val_map is the highest so far
    if val_map > best_val_map:
        best_val_map = val_map
        torch.save(checkpoint, os.path.join(directory, "best_model.pth"))
    
    return best_val_map
        


def save_epoch_data(directory, data):
    """
    Save training statistics for each epoch in a text file.

    Parameters:
    - directory (str): Path to the directory.
    - data (dict): Contains data on metrics such as epoch, losses, and validation metrics.
    """
    log_file_path = os.path.join(directory, "training_log.txt")
    
    lines = [
        f"Epoch {data['epoch']} | Time: {data['time_elapsed'][0]}m {data['time_elapsed'][1]}s | LR: {data['learning_rate']:.10f}\n",
        f"Train Loss: {data['train_loss']:.4f} | Classifier: {data['classifier_loss']:.4f} | Box Reg: {data['box_reg_loss']:.4f}\n",
        f"Objectness: {data['objectness_loss']:.4f} | RPN Box Reg: {data['rpn_box_reg_loss']:.4f}\n",
        f"Validation Metrics: | mAP@IoU=0.50:0.95: {data['val_metrics'][0]:.4f} | mAP@IoU=0.50: {data['val_metrics'][1]:.4f} | mAP@IoU=0.75: {data['val_metrics'][2]:.4f}\n",
        f"Small mAP: {data['val_metrics'][3]:.4f} | Medium mAP: {data['val_metrics'][4]:.4f} | Large mAP: {data['val_metrics'][5]:.4f}\n\n"
    ]
    with open(log_file_path, "a") as log_file:
        log_file.writelines(lines)


# Functions Data Augmentation

In [ ]:
class Compose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target

class RandomHorizontalFlip(object):
    def __init__(self, prob):
        self.prob = prob

    def __call__(self, image, target):
        # Check image type
        if not isinstance(image, (torch.Tensor, Image.Image)):
            raise TypeError(f"Unsupported image type: {type(image)}. Expected torch.Tensor or PIL.Image.")
        
        if torch.rand(1) < self.prob:
            if isinstance(image, torch.Tensor):
                width = image.shape[-1]
                image = F.hflip(image)
            else:
                width, _ = image.size
                image = F.hflip(image)
            
            # Flip bounding boxes
            bbox = target["boxes"]
            bbox[:, [0, 2]] = width - bbox[:, [2, 0]]  # Flip x-coordinates
            target["boxes"] = bbox
        return image, target

class RandomBrightnessCont(object):
    def __init__(self, brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5):
        self.color_jitter = ColorJitter(brightness, contrast, saturation, hue)
        self.p = p

    def __call__(self, image, target):
        if random.random() < self.p:
            image = self.color_jitter(image)
        return image, target

class RandomBlur(object):
    def __init__(self, kernel_size=3, p=0.5):
        self.blur = GaussianBlur(kernel_size)
        self.p = p

    def __call__(self, image, target):
        if random.random() < self.p:
            image = self.blur(image)
        return image, target

import math
import torch
import torchvision.transforms.functional as F
from PIL import Image

class RandomRotate(object):
    def __init__(self, angle_range=10, p=0.5):
        self.angle_range = angle_range
        self.p = p

    def __call__(self, image, target):
        if random.random() >= self.p:
            return image, target

        angle = random.uniform(-self.angle_range, self.angle_range)
        original_width, original_height = self._get_image_size(image)

        # Rotate image with expansion to get new dimensions
        image_pil = image if isinstance(image, Image.Image) else F.to_pil_image(image)
        image_pil_rotated = F.rotate(image_pil, angle, expand=True)
        new_width, new_height = image_pil_rotated.size

        # Convert back to tensor if needed
        image = F.to_tensor(image_pil_rotated) if isinstance(image, torch.Tensor) else image_pil_rotated

        # Rotate bounding boxes
        boxes = target['boxes']
        if len(boxes) == 0:
            return image, target

        # Compute rotation matrix with expansion offset
        cx_orig = original_width / 2
        cy_orig = original_height / 2

        # Calculate expansion offset (min_x, min_y)
        corners_original = torch.tensor([
            [0, 0],
            [original_width, 0],
            [original_width, original_height],
            [0, original_height]
        ])
        corners_rotated = self._rotate_points(corners_original, -angle, (cx_orig, cy_orig))
        min_x = corners_rotated[:, 0].min()
        min_y = corners_rotated[:, 1].min()

        # Rotate and translate box corners
        boxes_rotated = []
        for box in boxes:
            x1, y1, x2, y2 = box
            corners = torch.tensor([
                [x1, y1], [x2, y1], [x2, y2], [x1, y2]
            ])
            corners_rot = self._rotate_points(corners, -angle, (cx_orig, cy_orig))
            corners_rot -= torch.tensor([[min_x, min_y]])  # Adjust for expansion

            # Clamp to new image bounds
            x_min = max(0.0, corners_rot[:, 0].min().item())
            y_min = max(0.0, corners_rot[:, 1].min().item())
            x_max = min(new_width, corners_rot[:, 0].max().item())
            y_max = min(new_height, corners_rot[:, 1].max().item())

            if x_max > x_min and y_max > y_min:
                boxes_rotated.append([x_min, y_min, x_max, y_max])

        target['boxes'] = torch.tensor(boxes_rotated, dtype=torch.float32) if boxes_rotated else torch.zeros((0, 4), dtype=torch.float32)
        return image, target

    def _rotate_points(self, points, angle, center):
        angle_rad = math.radians(angle)
        cos_theta = math.cos(angle_rad)
        sin_theta = math.sin(angle_rad)
        cx, cy = center

        # Translate points to origin
        translated = points - torch.tensor([[cx, cy]])

        # Apply rotation
        x_rot = translated[:, 0] * cos_theta - translated[:, 1] * sin_theta
        y_rot = translated[:, 0] * sin_theta + translated[:, 1] * cos_theta

        # Translate back
        rotated_points = torch.stack([x_rot + cx, y_rot + cy], dim=1)
        return rotated_points

    def _get_image_size(self, image):
        if isinstance(image, torch.Tensor):
            return image.shape[-1], image.shape[-2]
        elif isinstance(image, Image.Image):
            return image.size
        else:
            raise TypeError("Unsupported image type.")
    
class RandomZoom(object):
    def __init__(self, zoom_range=(1.0, 2.0), p=0.5):
        self.zoom_range = zoom_range
        self.p = p

    def __call__(self, image, target):
        if random.random() < self.p:
            boxes = target['boxes']
            if len(boxes) == 0:
                return image, target

            box_idx = random.randint(0, len(boxes) - 1)
            x1, y1, x2, y2 = boxes[box_idx].numpy()

            width, height = image.size
            center_x, center_y = (x1 + x2) / 2, (y1 + y2) / 2
            box_width, box_height = x2 - x1, y2 - y1

            zoom_factor = random.uniform(*self.zoom_range)
            crop_width = box_width / zoom_factor
            crop_height = box_height / zoom_factor

            crop_x1 = max(0, center_x - crop_width / 2)
            crop_y1 = max(0, center_y - crop_height / 2)
            crop_x2 = min(width, center_x + crop_width / 2)
            crop_y2 = min(height, center_y + crop_height / 2)

            image = image.crop((int(crop_x1), int(crop_y1), int(crop_x2), int(crop_y2)))
            target['boxes'][:, [0, 2]] -= crop_x1
            target['boxes'][:, [1, 3]] -= crop_y1
            target['boxes'][:, [0, 2]] = target['boxes'][:, [0, 2]].clamp(0, crop_x2 - crop_x1)
            target['boxes'][:, [1, 3]] = target['boxes'][:, [1, 3]].clamp(0, crop_y2 - crop_y1)

        return image, target

    
def get_augmented_transform(train):
    """
    Get transform pipeline with augmentations for training or validation
    """
    transforms = []
    
    if train:
        # Applies a series of data augmentations specifically for the training set
        transforms.extend([
            RandomHorizontalFlip(0.5),  # Horizontally flips the image with a 50% probability
            RandomBrightnessCont(  # Adjusts brightness, contrast, saturation, and hue with specified ranges
                brightness=0.3, 
                contrast=0.4, 
                saturation=0.5, 
                hue=0.5, 
                p=0.5  # Applies these adjustments with a 50% probability
            ),
            RandomBlur(kernel_size=3, p=0.5),  # Applies Gaussian blur with a kernel size of 3, 30% chance
            RandomRotate(angle_range=50, p=0.5),  # Rotates the image by -10 to +10 degrees, 30% chance
            RandomZoom(zoom_range=(0.05, 0.99), p=0.6)  # Zooms the image by a factor between 0.05 and 0.9, 60% chance
        ])

    # Converts the image to a tensor for model input
    transforms.append(ToTensor())
    
    return Compose(transforms)

def visualize_augmentations(dataset, num_samples=5):
    fig, axes = plt.subplots(num_samples, 2, figsize=(12, 4 * num_samples))
    
    for i in range(num_samples):
        idx = random.randint(0, len(dataset) - 1)
        orig_img, orig_target = dataset[idx]
        
        if isinstance(orig_img, torch.Tensor):
            orig_img_np = orig_img.permute(1, 2, 0).numpy()
        else:
            orig_img_np = np.array(orig_img)
        
        axes[i, 0].imshow(orig_img_np)
        axes[i, 0].set_title('Original')
        
        aug_img, aug_target = dataset[idx]
        aug_img_np = aug_img.permute(1, 2, 0).numpy() if isinstance(aug_img, torch.Tensor) else np.array(aug_img)
        
        axes[i, 1].imshow(aug_img_np)
        axes[i, 1].set_title('Augmented')
    
    plt.tight_layout()
    plt.show()


In [ ]:
#let's make a function to visualise the data augmentation
dataset = HazmatDataset('data/data_faster_rcnn/train', 'data/data_faster_rcnn/train/annotations/instances_train.json', get_augmented_transform(train=True))
visualize_augmentations(dataset, num_samples=20)

In [ ]:
import matplotlib.pyplot as plt
import random
import numpy as np
import torch
from matplotlib.patches import Rectangle
dataset = HazmatDataset('data/data_faster_rcnn/train', 'data/data_faster_rcnn/train/annotations/instances_train.json', get_augmented_transform(train=True))
def visualize_augmentations(
    dataset, 
    num_samples=5, 
    bbox_format='xyxy', 
    denormalize_boxes=False, 
    box_color='red', 
    line_width=2, 
    get_boxes=lambda target: target.get('boxes', []), 
    denormalize_img=None, 
    seed=None,
    figsize=(15, 8)
):
    """
    Visualize original and augmented images with annotations.
    
    Parameters:
        dataset: Dataset object returning tuples (image, target).
        num_samples: Number of sample pairs to display.
        bbox_format: Bounding box format ('xyxy' or 'xywh').
        denormalize_boxes: Whether to denormalize box coordinates.
        box_color: Color for bounding boxes.
        line_width: Line width for bounding boxes.
        get_boxes: Function to extract boxes from target.
        denormalize_img: Tuple (mean, std) to reverse image normalization.
        seed: Random seed for reproducibility.
        figsize: Figure size.
    """
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
    
    fig, axes = plt.subplots(num_samples, 2, figsize=figsize)
    fig.suptitle('Original vs Augmented Images with Annotations', fontsize=16, y=1.02)
    
    # Handle single sample case
    if num_samples == 1:
        axes = axes.reshape(1, -1)
    
    for i in range(num_samples):
        idx = random.randint(0, len(dataset) - 1)
        
        original_dataset = HazmatDataset('data/data_faster_rcnn/test', 'data/data_faster_rcnn/test/annotations/instances_test.json', get_augmented_transform(train=False))
        # Get data pairs
        orig_img, orig_target = original_dataset[idx]
        aug_img, aug_target = dataset[idx]
        
        # Process original image
        orig_img_np = _process_image(orig_img, denormalize_img)
        _draw_image_and_boxes(axes[i, 0], orig_img_np, orig_target, 
                             f'Sample {i+1} - Original', 
                             get_boxes, bbox_format, denormalize_boxes, 
                             box_color, line_width)
        
        # Process augmented image
        aug_img_np = _process_image(aug_img, denormalize_img)
        _draw_image_and_boxes(axes[i, 1], aug_img_np, aug_target, 
                             f'Sample {i+1} - Augmented', 
                             get_boxes, bbox_format, denormalize_boxes, 
                             box_color, line_width)
        
        # Clean up axes
        axes[i, 0].axis('off')
        axes[i, 1].axis('off')
    
    plt.tight_layout()
    plt.show()

def _process_image(img, denormalize_params):
    """Convert tensor to numpy and denormalize if needed"""
    if isinstance(img, torch.Tensor):
        img_np = img.permute(1, 2, 0).numpy()
    else:
        img_np = np.array(img)
    
    if denormalize_params is not None:
        mean, std = denormalize_params
        img_np = img_np * std + mean
        img_np = np.clip(img_np, 0, 1)
    
    return img_np

def _draw_image_and_boxes(ax, img_np, target, title, 
                         get_boxes, bbox_format, denormalize_boxes,
                         box_color, line_width):
    """Helper to draw image and bounding boxes"""
    ax.imshow(img_np)
    ax.set_title(title)
    
    boxes = get_boxes(target)
    if len(boxes) == 0:
        return
    
    h, w = img_np.shape[:2]
    boxes = np.array(boxes)
    
    if denormalize_boxes:
        if bbox_format == 'xyxy':
            boxes *= np.array([w, h, w, h])
        elif bbox_format == 'xywh':
            x_center, y_center = boxes[:,0] * w, boxes[:,1] * h
            width, height = boxes[:,2] * w, boxes[:,3] * h
            x1 = x_center - width/2
            y1 = y_center - height/2
            boxes = np.stack([x1, y1, width, height], axis=1)
            bbox_format = 'xywh'  # Update format for conversion
    
    # Convert all boxes to xyxy format
    if bbox_format == 'xywh':
        boxes[:,0] = boxes[:,0] - boxes[:,2]/2
        boxes[:,1] = boxes[:,1] - boxes[:,3]/2
        boxes[:,2] = boxes[:,0] + boxes[:,2]
        boxes[:,3] = boxes[:,1] + boxes[:,3]
    
    for box in boxes:
        x1, y1, x2, y2 = box
        rect = Rectangle(
            (x1, y1), x2 - x1, y2 - y1,
            linewidth=line_width,
            edgecolor=box_color,
            facecolor='none'
        )
        ax.add_patch(rect)

dataset = HazmatDataset('data/data_faster_rcnn/test', 'data/data_faster_rcnn/test/annotations/instances_test.json', get_augmented_transform(train=True))
# Visualize augmentations
visualize_augmentations(
    dataset, 
    num_samples=10, 
    bbox_format='xyxy', 
    denormalize_boxes=False, 
    box_color='blue', 
    line_width=2, 
    get_boxes=lambda target: target.get('boxes', []), 
    figsize=(30, 30)
)


In [ ]:
import json
import os
import copy
import cv2
import numpy as np
from PIL import Image

def adjust_bbox_for_transforms(original_bbox, img_width, img_height, transforms_applied):
    """
    Adjust bounding box coordinates based on applied augmentations.
    """
    x, y, w, h = original_bbox
    x1, y1 = x, y
    x2, y2 = x + w, y + h

    for transform in transforms_applied:
        if transform["name"] == "RandomHorizontalFlip" and transform["applied"]:
            # Flip coordinates horizontally
            x1 = img_width - x2
            x2 = img_width - x
            x, w = x1, x2 - x1

        elif transform["name"] == "RandomZoom" and transform["applied"]:
            # Zoom-specific adjustments
            zoom_factor = transform["zoom_factor"]
            new_width = int(img_width * zoom_factor)
            new_height = int(img_height * zoom_factor)
            
            # Calculate crop coordinates (assuming center crop)
            left = (new_width - img_width) // 2
            top = (new_height - img_height) // 2
            right = left + img_width
            bottom = top + img_height
            
            # Adjust coordinates for zoom and crop
            x1 = max(0, x1 * zoom_factor - left)
            y1 = max(0, y1 * zoom_factor - top)
            x2 = min(img_width, x2 * zoom_factor - left)
            y2 = min(img_height, y2 * zoom_factor - top)
            
            x, y, w, h = x1, y1, x2 - x1, y2 - y1

    return [x, y, w, h]

In [ ]:
import json
import os
import copy
import torch
from PIL import Image
from tqdm import tqdm  # Added tqdm import

def export_augmented_dataset(
    original_img_dir,
    original_json_path,
    output_dir,
    num_augmentations=3
):
    # Load original COCO data
    with open(original_json_path, 'r') as f:
        coco_data = json.load(f)
    
    # Create output directories
    os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "annotations"), exist_ok=True)

    # Initialize new dataset structure
    new_data = {
        "images": [],
        "annotations": [],
        "categories": coco_data["categories"]
    }

    # Track IDs
    max_img_id = max(img["id"] for img in coco_data["images"]) if coco_data["images"] else 0
    max_ann_id = max(ann["id"] for ann in coco_data["annotations"]) if coco_data["annotations"] else 0

    # Prepare augmentation transforms (without ToTensor())
    augmentation_pipeline = get_augmented_transform(train=True)
    # Remove ToTensor() from pipeline for image saving
    augmentation_pipeline.transforms = [t for t in augmentation_pipeline.transforms 
                                      if not isinstance(t, ToTensor)]

    # Main progress bar for images
    for orig_img_info in tqdm(coco_data["images"], desc="Processing images", unit="img"):
        # Load original image
        img_path = os.path.join(original_img_dir, orig_img_info["file_name"])
        original_image = Image.open(img_path).convert("RGB")
        orig_width, orig_height = original_image.size

        # Get corresponding annotations
        original_annots = [ann for ann in coco_data["annotations"] 
                         if ann["image_id"] == orig_img_info["id"]]

        # Save original image and annotations to new dataset
        original_output_path = os.path.join(output_dir, "images", orig_img_info["file_name"])
        original_image.save(original_output_path)
        new_data["images"].append(copy.deepcopy(orig_img_info))
        for ann in original_annots:
            new_ann = copy.deepcopy(ann)
            new_data["annotations"].append(new_ann)

        # Convert COCO bboxes to x1y1x2y2 format
        boxes = []
        for ann in original_annots:
            x, y, w, h = ann["bbox"]
            boxes.append([x, y, x + w, y + h])

        # Create target dictionary
        original_target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32),
            "labels": torch.tensor([ann["category_id"] for ann in original_annots], dtype=torch.int64),
            "iscrowd": torch.tensor([ann["iscrowd"] for ann in original_annots], dtype=torch.int64)
        }

        # Augmentation progress bar
        for aug_idx in tqdm(range(num_augmentations), desc="Augmenting", leave=False, unit="aug"):
            # Create copies for augmentation
            aug_image = original_image.copy()
            aug_target = copy.deepcopy(original_target)

            # Apply augmentation pipeline
            for transform in augmentation_pipeline.transforms:
                aug_image, aug_target = transform(aug_image, aug_target)

            # Get new dimensions from PIL Image
            aug_width, aug_height = aug_image.size

            # Generate new filename
            base_name = os.path.splitext(orig_img_info["file_name"])[0]
            new_filename = f"{base_name}_aug{aug_idx}.jpg"
            new_img_path = os.path.join(output_dir, "images", new_filename)
            aug_image.save(new_img_path)

            # Create new image entry
            max_img_id += 1
            new_img_info = {
                "id": max_img_id,
                "file_name": new_filename,
                "width": aug_width,
                "height": aug_height
            }
            new_data["images"].append(new_img_info)

            # Process annotations
            valid_boxes = []
            for box, label, iscrowd in zip(aug_target["boxes"].numpy(),
                                         aug_target["labels"].numpy(),
                                         aug_target["iscrowd"].numpy()):
                # Convert back to COCO format
                x1, y1, x2, y2 = box
                w = x2 - x1
                h = y2 - y1

                # Filter invalid boxes
                if w > 0 and h > 0 and x1 < aug_width and y1 < aug_height:
                    valid_boxes.append({
                        "id": max_ann_id + 1,
                        "image_id": max_img_id,
                        "category_id": int(label),
                        "bbox": [float(x1), float(y1), float(w), float(h)],
                        "area": float(w * h),
                        "iscrowd": int(iscrowd)
                    })
                    max_ann_id += 1

            new_data["annotations"].extend(valid_boxes)

    # Save new annotations
    output_json_path = os.path.join(output_dir, "annotations", "instances_augmented.json")
    with open(output_json_path, 'w') as f:
        json.dump(new_data, f, indent=2)

    print(f"\nExported {len(new_data['images'])} images with {len(new_data['annotations'])} annotations")

In [ ]:
import json
import os
import copy
import random
import torch
from PIL import Image
import numpy as np

def export_augmented_dataset(
    original_img_dir,
    original_json_path,
    output_dir,
    num_augmentations=3
):
    # Load original COCO data
    with open(original_json_path, 'r') as f:
        coco_data = json.load(f)
    
    # Create output directories
    os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "annotations"), exist_ok=True)

    # Initialize new dataset structure
    new_data = {
        "images": [],
        "annotations": [],
        "categories": coco_data["categories"]
    }

    # Track IDs
    max_img_id = max(img["id"] for img in coco_data["images"]) if coco_data["images"] else 0
    max_ann_id = max(ann["id"] for ann in coco_data["annotations"]) if coco_data["annotations"] else 0

    # Prepare augmentation transforms (without ToTensor())
    augmentation_pipeline = get_augmented_transform(train=True)
    # Remove ToTensor() from pipeline for image saving
    augmentation_pipeline.transforms = [t for t in augmentation_pipeline.transforms 
                                      if not isinstance(t, ToTensor)]

    for orig_img_info in coco_data["images"]:
        # Load original image
        img_path = os.path.join(original_img_dir, orig_img_info["file_name"])
        original_image = Image.open(img_path).convert("RGB")
        orig_width, orig_height = original_image.size

        # Get corresponding annotations
        original_annots = [ann for ann in coco_data["annotations"] 
                         if ann["image_id"] == orig_img_info["id"]]

        # Save original image and annotations to new dataset
        original_output_path = os.path.join(output_dir, "images", orig_img_info["file_name"])
        original_image.save(original_output_path)
        new_data["images"].append(copy.deepcopy(orig_img_info))
        for ann in original_annots:
            new_ann = copy.deepcopy(ann)
            new_data["annotations"].append(new_ann)

        # Convert COCO bboxes to x1y1x2y2 format
        boxes = []
        for ann in original_annots:
            x, y, w, h = ann["bbox"]
            boxes.append([x, y, x + w, y + h])

        # Create target dictionary
        original_target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32),
            "labels": torch.tensor([ann["category_id"] for ann in original_annots], dtype=torch.int64),
            "iscrowd": torch.tensor([ann["iscrowd"] for ann in original_annots], dtype=torch.int64)
        }

        for aug_idx in range(num_augmentations):
            # Create copies for augmentation
            aug_image = original_image.copy()
            aug_target = copy.deepcopy(original_target)

            # Apply augmentation pipeline
            for transform in augmentation_pipeline.transforms:
                aug_image, aug_target = transform(aug_image, aug_target)

            # Get new dimensions from PIL Image
            aug_width, aug_height = aug_image.size

            # Generate new filename
            base_name = os.path.splitext(orig_img_info["file_name"])[0]
            new_filename = f"{base_name}_aug{aug_idx}.jpg"
            new_img_path = os.path.join(output_dir, "images", new_filename)
            aug_image.save(new_img_path)

            # Create new image entry
            max_img_id += 1
            new_img_info = {
                "id": max_img_id,
                "file_name": new_filename,
                "width": aug_width,
                "height": aug_height
            }
            new_data["images"].append(new_img_info)

            # Process annotations
            valid_boxes = []
            for box, label, iscrowd in zip(aug_target["boxes"].numpy(),
                                         aug_target["labels"].numpy(),
                                         aug_target["iscrowd"].numpy()):
                # Convert back to COCO format
                x1, y1, x2, y2 = box
                w = x2 - x1
                h = y2 - y1

                # Filter invalid boxes
                if w > 0 and h > 0 and x1 < aug_width and y1 < aug_height:
                    valid_boxes.append({
                        "id": max_ann_id + 1,
                        "image_id": max_img_id,
                        "category_id": int(label),
                        "bbox": [float(x1), float(y1), float(w), float(h)],
                        "area": float(w * h),
                        "iscrowd": int(iscrowd)
                    })
                    max_ann_id += 1

            new_data["annotations"].extend(valid_boxes)

    # Save new annotations
    output_json_path = os.path.join(output_dir, "annotations", "instances_augmented.json")
    with open(output_json_path, 'w') as f:
        json.dump(new_data, f, indent=2)

    print(f"Exported {len(new_data['images'])} images with {len(new_data['annotations'])} annotations")

In [ ]:
export_augmented_dataset(
    original_img_dir="data/data_faster_rcnn/train/images",
    original_json_path="data/data_faster_rcnn/train/annotations/instances_train.json",
    output_dir="data/data_faster_rcnn/augmented_examplev43",
    num_augmentations=4  # Number of augmented versions per image
)

In [ ]:
# get function to see if annotations are right
import cv2
# read the json files
def check_images_in_annotations(destination_dir, destination_dir_for_images, max_images=10):
    with open(destination_dir) as f:
        data = json.load(f)
        annotations = data["annotations"]
        #shffle
        random.shuffle(annotations)
        for annotation in annotations[:max_images]:
            image_id = annotation["image_id"]
            images = data["images"]
            image = next((image for image in images if image["id"] == image_id), None)

            if image is None:
                print(f"Image not found for annotation: {annotation}")
            else:
                # show the image with bounding box
                # Read the image
                path_im = destination_dir_for_images + "/"+ image["file_name"]
                file_name = image["file_name"]
                image = cv2.imread(path_im)
                if image is None:
                    print(f"Image not found: {path_im}")
                    continue

                # Convert BGR (OpenCV format) to RGB (Matplotlib format)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                # Extract bounding box coordinates
                x, y, w, h = annotation["bbox"]
                x, y, w, h = int(x), int(y), int(w), int(h)

                # Plot the image
                plt.figure(figsize=(8, 8))
                plt.imshow(image)

                # Draw the bounding box
                plt.gca().add_patch(plt.Rectangle((x, y), w, h, edgecolor='green', facecolor='none', linewidth=2))

                
                # Display the image with the bounding box
                plt.title(f"Image file: {file_name}")
                plt.axis("off")
                plt.show()

annotations_file = "data/data_faster_rcnn/augmented_examplev12/annotations/instances_augmented.json"
images_dir = "data/data_faster_rcnn/augmented_examplev12/images"
check_images_in_annotations(annotations_file,images_dir, max_images=150)


In [ ]:
import torch
#see if torch is available
print(torch.__version__)

In [ ]:
!nvidia-smi

In [ ]:
!kill -9 25180

In [ ]:
#empty cuda cache
torch.cuda.empty_cache()
device = torch.device('cuda')
print(f"Training model on {device}")
from torchvision.models.detection.backbone_utils import mobilenet_backbone
# Create datasets
train_dataset = HazmatDataset(
    data_dir='data/data_faster_rcnn/augmented_trainv2',
    annotations_file='data/data_faster_rcnn/augmented_trainv2/annotations/instances_augmented.json',
    transforms=get_augmented_transform(train=False)
)

val_dataset = HazmatDataset(
    data_dir='data/data_faster_rcnn/val',
    annotations_file='data/data_faster_rcnn/val/annotations/instances_val.json',
    transforms=get_augmented_transform(train=False)
)

# amount of cpu cores
workers = 0
batchsize = 4
# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=batchsize,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=workers,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batchsize,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=workers,
    pin_memory=True
)

# Initialize model
num_classes = 2  # hazmat code and background
backbone = mobilenet_backbone('mobilenet_v2', pretrained=True)
# Create ResNet-101 backbone with FPN
# backbone = resnet_fpn_backbone('resnet101', pretrained=True)

# Define anchor generator for FPN
anchor_generator = AnchorGenerator(
    sizes=((32,), (64,), (128,), (256,), (512,)),
    aspect_ratios=((0.5, 1.0, 2.0),) * 5
)

# Multi-scale RoI pooling for FPN
roi_pooler = MultiScaleRoIAlign(
    featmap_names=['0', '1', '2', '3', '4'],
    output_size=7,
    sampling_ratio=2
)

print("initializing model...")
# Initialize Faster R-CNN with ResNet-101-FPN
model = FasterRCNN(
    backbone=backbone,
    num_classes=num_classes,
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler
)

# Move model to device
model.to(device)

# training

In [ ]:
scaler = GradScaler()

# Initialize optimizer and scheduler
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Training loop
num_epochs = 1
train_metrics_map = []
best_val_map = float('-inf')

print("Starting training...")

# Create directory to store models and logs
directory_finetuned_model = create_directory()


for epoch in range(num_epochs):
    best_val_map = train_model(
        directory=directory_finetuned_model, 
        model=model, optimizer=optimizer, train_loader=train_loader, device=device, 
        train_metrics_list=train_metrics_map, best_val_map=best_val_map, lr_scheduler=lr_scheduler, 
        val_loader=val_loader, coco_val=coco_val, scaler=scaler, epoch=epoch
    )



In [ ]:
from collections import OrderedDict
device = torch.device('cuda:0')
print(f"Training model on {device}")

data_dir_train = "augmented_examplev12"
annotations_d = "augmented"

    
# Create datasets
train_dataset = HazmatDataset(
    data_dir=f'data/data_faster_rcnn/{data_dir_train}',
    annotations_file=f'data/data_faster_rcnn/{data_dir_train}/annotations/instances_{annotations_d}.json',
    transforms=get_augmented_transform(train=False)
)

val_dataset = HazmatDataset(
    data_dir='data/data_faster_rcnn/val',
    annotations_file='data/data_faster_rcnn/val/annotations/instances_val.json',
    transforms=get_augmented_transform(train=False)
)

# Set the percentage of the training dataset to use (e.g. 0.x to 1)
train_percentage = 1

# Create a subset of the training dataset
train_dataset_subset = create_subset(train_dataset, train_percentage)

# Set the percentage of the val dataset to use (e.g. 0.x to 1)
val_percentage = 1

# Create a subset of the training dataset
val_dataset_subset = create_subset(val_dataset, val_percentage)

# amount of cpu cores
workers = 2

# Create data loaders
train_loader = DataLoader(
    train_dataset_subset,
    batch_size=8,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=workers,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset_subset,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=workers,
    pin_memory=True
)

# Initialize model
num_classes = 2  # hazmat code and background

# Create ResNet-101 backbone with FPN
backbone = resnet_fpn_backbone('resnet101', pretrained=True)

# Define anchor generator for FPN
anchor_generator = AnchorGenerator(
    sizes=((32,), (64,), (128,), (256,), (512,)),
    aspect_ratios=((0.5, 1.0, 2.0),) * 5
)

# Multi-scale RoI pooling for FPN
roi_pooler = MultiScaleRoIAlign(
    featmap_names=['0', '1', '2', '3', '4'],
    output_size=7,
    sampling_ratio=2
)

print("initializing model...")
# Initialize Faster R-CNN with ResNet-101-FPN
model = FasterRCNN(
    backbone=backbone,
    num_classes=num_classes,
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler
)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# Move model to device
model.to(device)

In [ ]:
# Load the model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
directory_finetuned_model = os.path.join("data", "models")
device = torch.device('cuda')
model_path = os.path.join(directory_finetuned_model, 'best_model.pth')
checkpoint = torch.load(model_path, map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
val_map = checkpoint['val_map']
epoch = checkpoint['epoch']
#latest
# latest_model_path = os.path.join(directory_finetuned_model, 'latest_model.pth')
# checkpoint_latest = torch.load(latest_model_path, map_location=device)
# val_map_latest = checkpoint_latest['val_map']
# epoch_latest = checkpoint_latest['epoch']

model.load_state_dict(checkpoint['model_state_dict'])
model.eval()  # Set the model to evaluation mode

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# model = torch.load('data/models/best_model.pth', map_location=torch.device('cpu'))
def load_image(image_path, transforms=None):
    image = Image.open(image_path).convert('RGB')
    if transforms:
        for transform in transforms:
            image, _ = transform(image, target=None)  # No target during inference
    return image

def get_transform(train):
    transforms = []
    # Convert PIL image to tensor
    transforms.append(ToTensor())
    if train:
        # Add training augmentations here if needed
        transforms.append(RandomHorizontalFlip(0.5))
    return transforms

# Define preprocessing transforms
test_transforms = get_transform(train=False)

# Load the image
image_path = 'images/hazard_plate.jpg'  # Replace with your image path
image = load_image(image_path, transforms=test_transforms)
image = image.to(device)
# Wrap the image in a list as the model expects a batch
with torch.no_grad():
    predictions = model([image])

In [ ]:
def draw_predictions(image, predictions, threshold=0.5, classes=['background', 'hazmat']):
    # Convert image from tensor to numpy array
    image = image.cpu().permute(1, 2, 0).numpy()
    image = np.clip(image * 255, 0, 255).astype(np.uint8)
    
    boxes = predictions[0]['boxes'].cpu().numpy()
    labels = predictions[0]['labels'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()
    
    # Filter predictions based on confidence threshold
    keep = scores >= threshold
    boxes = boxes[keep]
    labels = labels[keep]
    scores = scores[keep]
    
    fig, ax = plt.subplots(1, figsize=(12, 9))
    ax.imshow(image)
    
    for box, label, score in zip(boxes, labels, scores):
        if label == 1:  # Only plot hazmat codes
            x1, y1, x2, y2 = box
            rect = plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, color='blue', linewidth=2)
            ax.add_patch(rect)
            label_name = classes[label]
            ax.text(x1, y1, f'{label_name}: {score:.2f}', color='white', backgroundcolor='blue', fontsize=12)
    
    plt.show()

In [ ]:
def predict_image(image_path, threshold=0.5):
    # List of class names
    classes = ['background', 'hazmat']
    
    # Load the image
    image = load_image(image_path, transforms=test_transforms)
    image = image.to(device)
    
    # Wrap the image in a list as the model expects a batch
    with torch.no_grad():
        predictions = model([image])
    
    # Filter predictions based on threshold
    boxes = predictions[0]['boxes'].cpu().numpy()
    labels = predictions[0]['labels'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()
    
    # Apply threshold filter
    keep = scores >= threshold
    boxes = boxes[keep]
    labels = labels[keep]
    scores = scores[keep]
    
    # Print the predictions
    if len(boxes) == 0:
        print("No predictions meet the threshold.")
    else:
        print("Predictions:")
        for label, score in zip(labels, scores):
            class_name = classes[label]
            print(f"  {class_name}: {score:.2f}")
        # Display the predictions
        draw_predictions(image, predictions, threshold=threshold, classes=classes)

In [ ]:
import os
import glob

# Function to predict all images in a directory (including subdirectories)
def predict_all_images(directory, threshold=0):
    image_extensions = ('*.jpg', '*.jpeg', '*.png', '*.webp')  # Add more extensions if needed
    image_files = []
    
    # Collect all images from the directory and subdirectories
    for ext in image_extensions:
        image_files.extend(glob.glob(os.path.join(directory, '**', ext), recursive=True))
    
    # Run predictions on each image
    for image_path in image_files:
        print(f"Predicting: {image_path}")
        predict_image(image_path, threshold=threshold)

# Call the function with your images folder
predict_all_images('images', threshold=0)


In [ ]:
predict_image('data/data_faster_rcnn/val/images/1690281365_00595.jpg', threshold=0.29)
predict_image('images/hazard_plate.jpg', threshold=0)
predict_image('images/un_numbers_test/1.webp', threshold=0)
predict_image('images/un_numbers_test/2.jpg', threshold=0)
predict_image('images/un_numbers_test/3.jpg', threshold=0)
predict_image('images/un_numbers_test/4.jpg', threshold=0)
predict_image('images/un_numbers_test/6.webp', threshold=0)
predict_image('images/un_numbers_test/7.jpg', threshold=0)
